# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 53 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2304.03871


extracting tarball to tmp_2304.03871...

 done.


/tmp/ipykernel_2024/1863104449.py:49: LatexWarning: 2304.03871 did not run properly
[Errno 2] No such file or directory: 'gs'
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2304.04154


extracting tarball to tmp_2304.04154...

 done.
Retrieving document from  https://arxiv.org/e-print/2304.04719


extracting tarball to tmp_2304.04719... done.


Found 73 bibliographic references in tmp_2304.04719/sample631.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2304.04719-b31b1b.svg)](https://arxiv.org/abs/arXiv:2304.04719) | **(SHERRY) JCMT-SCUBA2 High Redshift Bright Quasar Survey -- II: the  environment of z~6 quasars in sub-millimeter band**  |
|| Q. Li, et al. -- incl., <mark>E. Bañados</mark> |
|*Appeared on*| *2023-04-11*|
|*Comments*| *12 pages, 8 figures and 2 tables, accepted for publication in ApJ*|
|**Abstract**| The formation of the first supermassive black holes is expected to have occurred in some most pronounced matter and galaxy overdensities in the early universe. We have conducted a sub-mm wavelength continuum survey of 54 $z\sim6$ quasars using the Submillimeter Common-User Bolometre Array-2 (SCUBA2) on the James Clerk Maxwell Telescope (JCMT) to study the environments around $z \sim 6$ quasars. We identified 170 submillimeter galaxies (SMGs) with above 3.5$\sigma$ detections at 450 or 850 \um\, maps. Their FIR luminosities are 2.2 - 6.4 $\times$ 10$^{12} L_{\odot}$, and star formation rates are $\sim$ 400 - 1200 M$_{\odot}$ yr$^{-1}$. We also calculated the SMGs differential and cumulative number counts in a combined area of $\sim$ 620 arcmin$^2$. To a $4\sigma$ detection (at $\sim$ 5.5 mJy), SMGs overdensity is $0.68^{+0.21}_{-0.19}$($\pm0.19$), exceeding the blank field source counts by a factor of 1.68. We find that 13/54 quasars show overdensities (at $\sim$ 5.5 mJy) of $\delta_{SMG}\sim$ 1.5 - 5.4. The combined area of these 13 quasars exceeds the blank field counts with the overdensity to 5.5 mJy of \dsmg $\sim$ $2.46^{+0.64}_{-0.55}$($\pm0.25$) in the regions of $\sim$ 150 arcmin$^2$. However, the excess is insignificant on the bright end (e.g., 7.5 mJy). We also compare results with previous environmental studies of Lyman alpha emitters (LAEs) and Lyman-Break Galaxies (LBGs) on a similar scale. Our survey presents the first systematic study of the environment of quasars at $z\sim6$. The newly discovered SMGs provide essential candidates for follow-up spectroscopic observations to test whether they reside in the same large-scale structures as the quasars and search for protoclusters at an early epoch. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2304.04154-b31b1b.svg)](https://arxiv.org/abs/arXiv:2304.04154) | **Review of X-ray pulsar spacecraft autonomous navigation**  |
|| Y. Wang, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2023-04-11*|
|*Comments*| *has been accepted by Chinese Journal of Aeronautics*|
|**Abstract**| This article provides a review on X-ray pulsar-based navigation (XNAV). The review starts with the basic concept of XNAV, and briefly introduces the past, present and future projects concerning XNAV. This paper focuses on the advances of the key techniques supporting XNAV, including the navigation pulsar database, the X-ray detection system, and the pulse time of arrival estimation. Moreover, the methods to improve the estimation performance of XNAV are reviewed. Finally, some remarks on the future development of XNAV are provided. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2304.03871-b31b1b.svg)](https://arxiv.org/abs/arXiv:2304.03871) | **KMT-2021-BLG-2010Lb, KMT-2022-BLG-0371Lb, and KMT-2022-BLG-1013Lb: Three  microlensing planets detected via partially covered signals**  |
|| C. Han, et al. -- incl., <mark>A. Gould</mark> |
|*Appeared on*| *2023-04-11*|
|*Comments*| *12 pages, 17 figures*|
|**Abstract**| We inspect 4 microlensing events KMT-2021-BLG-1968, KMT-2021-BLG-2010, KMT-2022-BLG-0371, and KMT-2022-BLG-1013, for which the light curves exhibit partially covered short-term central anomalies. We conduct detailed analyses of the events with the aim of revealing the nature of the anomalies. We test various models that can give rise to the anomalies of the individual events including the binary-lens (2L1S) and binary-source (1L2S) interpretations. Under the 2L1S interpretation, we thoroughly inspect the parameter space to check the existence of degenerate solutions, and if they exist, we test the feasibility of resolving the degeneracy. We find that the anomalies in KMT-2021-BLG-2010 and KMT-2022-BLG-1013 are uniquely defined by planetary-lens interpretations with the planet-to-host mass ratios of $q\sim 2.8\times 10^{-3}$ and $\sim 1.6\times 10^{-3}$, respectively. For KMT-2022-BLG-0371, a planetary solution with a mass ratio $q\sim 4\times 10^{-4}$ is strongly favored over the other three degenerate 2L1S solutions with different mass ratios based on the $\chi^2$ and relative proper motion arguments, and a 1L2S solution is clearly ruled out. For KMT-2021-BLG-1968, on the other hand, we find that the anomaly can be explained either by a planetary or a binary-source interpretation, making it difficult to firmly identify the nature of the anomaly. From the Bayesian analyses of the identified planetary events, we estimate that the masses of the planet and host are $(M_{\rm p}/M_{\rm J}, M_{\rm h}/M_\odot) = (1.07^{+1.15}_{-0.68}, 0.37^{+0.40}_{-0.23})$, $(0.26^{+0.13}_{-0.11}, 0.63^{+0.32}_{-0.28})$, and $(0.31^{+0.46}_{-0.16}, 0.18^{+0.28}_{-0.10})$ for KMT-2021-BLG-2010L, KMT-2022-BLG-0371L, and KMT-2022-BLG-1013L, respectively. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error [Errno 2] No such file or directory: 'gs'</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2304.04719.md
    + _build/html/tmp_2304.04719/figures/figure_1a.png
    + _build/html/tmp_2304.04719/figures/figure_4.png
    + _build/html/tmp_2304.04719/figures/figure_7.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\um}{\mum}$
$\newcommand{\ai}{\mbox{\normalfontÅ} }$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\cii}{[C\thinspace{\sc ii}]}$</div>



<div id="title">

# (SHERRY) JCMT-SCUBA2 High rEdshift bRight quasaR surveY - II: the environment of $z\sim6$ quasars in sub-millimeter band

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2304.04719-b31b1b.svg)](https://arxiv.org/abs/2304.04719)<mark>Appeared on: 2023-04-11</mark> -  _12 pages, 8 figures and 2 tables, accepted for publication in ApJ_

</div>
<div id="authors">

Q. Li, et al. -- incl., <mark>E. Bañados</mark>

</div>
<div id="abstract">

**Abstract:** The formation of the first supermassive black holes is expected to have occurred in some most pronounced matter and galaxy overdensities in the early universe. We have conducted a sub-mm wavelength continuum survey of 54 $z\sim6$ quasars using the Submillimeter Common-User Bolometre Array-2 (SCUBA2) on the James Clerk Maxwell Telescope (JCMT)to study the environments around $z \sim 6$ quasars. We identified 170 submillimeter galaxies (SMGs) with above 3.5 $\sigma$ detections at 450 or 850 $\um$ maps.Their FIR luminosities are 2.2 - 6.4 $\times$ 10 $^{12} L_{\odot}$ , and star formation rates are $\sim$ 400 - 1200 M $_{\odot}$ yr $^{-1}$ . We also calculated the SMGs differential and cumulative number counts in a combined area of $\sim$ 620 arcmin $^2$ . To a $4\sigma$ detection (at $\sim$ 5.5 mJy), SMGs overdensity is $0.68^{+0.21}_{-0.19}$ ( $\pm0.19$ ), exceeding the blank field source counts by a factor of 1.68. We find that 13/54 quasars show overdensities (at $\sim$ 5.5 mJy) of $\delta_{SMG}\sim$ 1.5 - 5.4. The combined area of these 13 quasars exceeds the blank field counts with the overdensity to 5.5 mJy of $\dsmg$ $\sim$ $2.46^{+0.64}_{-0.55}$ ( $\pm0.25$ ) in the regions of $\sim$ 150 arcmin $^2$ . However, the excess is insignificant on the bright end (e.g., 7.5 mJy). We also compare results with previous environmental studies of Lyman alpha emitters (LAEs) and Lyman-Break Galaxies (LBGs) on a similar scale. Our survey presents the first systematic study of the environment of quasars at $z\sim6$ . The newly discovered SMGs provide essential candidates for follow-up spectroscopic observations to test whether they reside in the same large-scale structures as the quasars and search for protoclusters at an early epoch.

</div>

<div id="div_fig1">

<img src="tmp_2304.04719/figures/figure_1a.png" alt="Fig3" width="100%"/>

**Figure 3. -** JCMT-SCUBA2 850$\um$  images of the surrounding regions around 54 $z \sim 6$ quasar with a beam size of FWHM = 15$\arcsec$.
The selected region is within 2.23 arcmins from the quasar optical position (which covers typical protocluster scales at $z \sim 6$) and has one sigma uncertainty of $<$ 1.5 mJy beam$^{-1}$ for each pixel. The contour levels are +3, +4, +5, +6, +7$\times$1.5 mJy beam$^{-1}$ for each map. The star indicates the quasar optical/NIR position. The star with the blue edge is the sub-mm detected quasars. The blue circles indicate the selected SMGs (with S/N$>$3.5 at 850um or 450um, see Table \ref{table2}).
The sources marked with `\#' are the SMGs used in the overdensity analysis with deboosted fluxes $>$ 5.5 mJy.
 (*figure_1*)

</div>
<div id="div_fig2">

<img src="tmp_2304.04719/figures/figure_4.png" alt="Fig6" width="100%"/>

**Figure 6. -** Example completeness and flux boosting estimates based on Monte Carlo source insertion. Left: Completeness as a function of input flux (S$_{in}$). The solid curve represents the best-fit function of *f(S$_{in*$) = [1+erf((S$_{in}$-A)/B)]/2} with best fitting parameters give in the text. Right: The ratio between input flux density and output flux density as a function of input flux. Error bars show 1$\sigma$ of 5000 trials. Solid curve represents the best-fit function of *f(S$_{in*$) =1+Aexp(-BS$_{in} ^C$)}.  ([Hatsukade, Kohno and Umehata 2016]()) . (*fig:completeness and boosting*)

</div>
<div id="div_fig3">

<img src="tmp_2304.04719/figures/figure_7.png" alt="Fig8" width="100%"/>

**Figure 8. -** Cumulative number counts of SMGs, in fields centered on (i) luminous high-redshift AGN: z $\sim$ 5 optically selected quasars (grey diamonds;  ([Priddey, Ivison and Isaak 2008]()) ); z $\sim$ 4 radio galaxies (grey squares;  ([Stevens, Ivison and Dunlop 2003]()) ); (ii) z $\sim$ 2 protoclusters (grey triangles;  ([Zhang, Zheng and Shi 2022](), [Cooke, Smail and Stach 2019]()) ); (iii) z $\sim$ 1 clusters (grey circles;  ([ and Best 2002](), [Webb, Yee and Ivison 2005]()) ) and (iv) $z \sim 6$ optically selected quasars (this paper, black points and red/blue diamonds).
The black line shows the blank-field number counts of SMGs in the previous survey, i.e., SCUBA2 survey for S2CLS field in [Geach, Dunlop and Halpern (2017)]().
The blue line is calculated from our survey's combined areas of all 54 quasars ($\sim$ 620 arcmin$^2$); the red line is from the combined areas of 13 overdense fields around the quasars ($\sim$ 150 arcmin$^2$).
The black-filled points indicate the three most overdense fields selected from our sample at $z \sim 6$.
Points in this work have been corrected for flux boosting and incompleteness. (*figure_7*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2304.04719"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

186  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

5  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
